Goal of this notebook:
 - Implement the other type of determing connections between vertices at a time:
     - There is only one edge between two vertices that represents connection.
     - Each edge stores a "log" of changes to a connection, which is just a list of (time, connected) pairs, where <time> specifies the time of the change, and <connected> specifies whether the two vertices were connected or disconnected at this time.
     - When querying the graph with a specific timestamp, look at a specific edge, then do a binary search on the "time" values of each element of the "log" list, and return whether the most recent change was a connection or a disconnection.

In [133]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

# Get the vertices of the graph as a list, and print them.

print(g.V().toList())

[v[81973256], v[24760], v[41005160]]


In [134]:
# Drop all vertices of the graph.
g.V().drop().iterate()
g.E().drop().iterate()

[['E'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [135]:
# Add two vertices that are to be connected

# Radio frequency over fiber
g.addV().property('name', 'RFoF').next()

# Analog-Digital converter
g.addV().property('name', 'ADC').next()

# Optical fiber
g.addV().property('name', 'OF').next()


v[81977352]

In [136]:
def get_next_smallest_index(val, lst) -> int:
    """
    Given a sorted list lst in increasing order and val where val is of the same type as all elements in lst, 
    do a binary search and return the lower bound on the index (if the exact value is not found).
    """

    l, r = 0, len(lst) - 1

    while l <= r:
        mid = l + (r - l) // 2 
        if val > lst[mid]:
            l = mid + 1
        elif val < lst[mid]:
            r = mid - 1
        else:
            return mid
    return l - 1

# lst = [1, 3, 5, 7]
# for i in range(10):
#     print(i, lst[get_next_smallest_index(i, lst)])


In [137]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>.
    Do so by sending a Gremlin query to find an edge labelled 'connected' between the two vertices. If one exists, get its "log" of changes to connections (which is just a list), and do a binary search to determine whether at <time>, the edge was active.
    """

    # Get the vertices associated with the names
    v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Get the "connection log" list of the edge that connects v1 and v2. If no edge exists, this should be an empty list.
    # The list should be of the format [[t, c], [t, c], ..., [t, c]], where t is the time of the change and c represents whether
    # the element was connected or disconnected at the time.
    connections = g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).values('connection_log').toList() 

    # Get a list of the times of the connections
    times = [c[0] for c in connections]

    index = get_next_smallest_index(time, times)

    if index == -1:
        return False
    else:
        return connections[index][1]

In [161]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, either add a new [time, connection] entry to the connection log of the 'connection' edge connecting the two vertices, or create a new edge if such an edge does not already exist.
    
    Preconditions:
         - time is greater than the times of all previous connections!

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # This prevents multiple entries of the same type.
    if connection != connected(name1, name2, time):

        # Get the vertices associated with the names
        v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

        print(g.V(v1).bothE('connection').filter(__.bothV().is_(v2)).fold().coalesce(
            __.unfold(), 
            __.addE('connection').from_(v1).to(v2)
        ).property(Cardinality.list_, 'connection_log', [time, connection]).valueMap().toList()) #.iterate()

In [162]:
# Set some connections.

rfof_adc = [(1, True), (3, False), (4, True), (5, False), (7, True)]
rfof_of = [(2, True), (3, False), (5, True), (6, False), (9, True)]

for (time, connection) in rfof_adc:
    set_connection(name1='RFoF', name2='ADC', time=time, connection=connection)

for (time, connection) in rfof_of:
    set_connection(name1='RFoF', name2='OF', time=time, connection=connection)

GremlinServerError: 500: org.janusgraph.graphdb.relations.CacheEdge cannot be cast to org.apache.tinkerpop.gremlin.structure.Vertex

In [163]:
# Go through each combination of element and see if they are connected at all times from 0 to 10.

import itertools

time_min = 0
time_max = 10

names = ['RFoF', 'ADC', 'OF']

combinations = list(itertools.combinations(names, 2))

for (name1, name2) in combinations:
    for time in range(time_min, time_max + 1):
        print(f"({name1}, {name2}) at {time}: {connected(name1, name2, time)}")

(RFoF, ADC) at 0: False
(RFoF, ADC) at 1: False
(RFoF, ADC) at 2: False
(RFoF, ADC) at 3: False
(RFoF, ADC) at 4: False
(RFoF, ADC) at 5: False
(RFoF, ADC) at 6: False
(RFoF, ADC) at 7: False
(RFoF, ADC) at 8: False
(RFoF, ADC) at 9: False
(RFoF, ADC) at 10: False
(RFoF, OF) at 0: False
(RFoF, OF) at 1: False
(RFoF, OF) at 2: False
(RFoF, OF) at 3: False
(RFoF, OF) at 4: False
(RFoF, OF) at 5: False
(RFoF, OF) at 6: False
(RFoF, OF) at 7: False
(RFoF, OF) at 8: False
(RFoF, OF) at 9: False
(RFoF, OF) at 10: False
(ADC, OF) at 0: False
(ADC, OF) at 1: False
(ADC, OF) at 2: False
(ADC, OF) at 3: False
(ADC, OF) at 4: False
(ADC, OF) at 5: False
(ADC, OF) at 6: False
(ADC, OF) at 7: False
(ADC, OF) at 8: False
(ADC, OF) at 9: False
(ADC, OF) at 10: False
